In [ ]:
!git clone https://github.com/datvodinh10/project-DD.git

In [1]:
import torch

In [ ]:
from src.model.trainer import Trainer

In [ ]:
config = {
    'transformer':{
        'embed_size': 256,
        'num_heads':4,
        'num_layers':2,
        'max_len': 10_000,
        'dropout':0.2,
        'bias':False,     
        'block_size':256,
        
    },
    'device':torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    'lr':1e-4,
    'batch_size':32,
    'n_iter':5
}

In [ ]:
trainer = Trainer(config      = config,
                  IMAGE_PATH  = "/content/new_train",
                  TARGET_PATH = "/content/gdrive/MyDrive/Handwritten_OCR/train_gt.txt")